In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to C:\Users\Kousik
[nltk_data]     C\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Kousik
[nltk_data]     C\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data=pd.read_csv(r"C:\Users\Kousik C\Downloads\megaGymDataset.csv")

In [3]:
data.head()


,Index,Title,Desc,Type,BodyPart,Equipment,Level
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate


In [4]:
data.shape

(2872, 7)

In [5]:
data.Level.value_counts()

Level
Intermediate    2416
Beginner         443
Expert            13
Name: count, dtype: int64

In [6]:
data.Equipment.value_counts()

Equipment
Body Only        1046
Dumbbell          516
Barbell           282
Other             249
Cable             226
Machine           173
Kettlebells       149
Bands              97
Medicine Ball      38
Exercise Ball      35
E-Z Curl Bar       22
Foam Roll           9
Name: count, dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2872 entries, 0 to 2871
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Index      2872 non-null   int64 
 1   Title      2872 non-null   object
 2   Desc       1353 non-null   object
 3   Type       2872 non-null   object
 4   BodyPart   2872 non-null   object
 5   Equipment  2842 non-null   object
 6   Level      2872 non-null   object
dtypes: int64(1), object(6)
memory usage: 157.2+ KB


In [8]:
data.BodyPart.value_counts()

BodyPart
Abdominals     662
Quadriceps     646
Shoulders      340
Chest          262
Biceps         168
Triceps        151
Lats           124
Hamstrings     121
Middle Back    118
Lower Back      97
Glutes          81
Calves          47
Forearms        31
Traps           24
Name: count, dtype: int64

In [12]:
data = data.dropna(subset=['Desc'])

In [9]:
data.Level.value_counts()

Level
Intermediate    2416
Beginner         443
Expert            13
Name: count, dtype: int64

In [10]:

lemmatizer = WordNetLemmatizer()
def tokenize(text):
    if isinstance(text, float) and np.isnan(text):
        return []
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens


In [11]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize, max_features=1000)
X = tfidf_vectorizer.fit_transform(data['Desc'])
y = data['BodyPart']

C:\Users\Kousik C\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
def welcome_message():
    return "Welcome to Fitty! Your personal gym assistant. How can I help you today?"

# Chatbot function
def fitty_chatbot(query):
    query_vectorized = tfidf_vectorizer.transform([query])
    prediction = rf_classifier.predict(query_vectorized)
    return prediction[0]


In [ ]:
def part_exercises(body_part,difficulty_level):
    body_parts = data['BodyPart'].unique()
    random_exercises = {}
    body_part_exercises = data[(data['BodyPart'] == body_part) & (data['Level'] == difficulty_level)]['Title']
    random_exercise = body_part_exercises.sample(n=1).values[0]
    random_exercises[body_part] = random_exercise
    return random_exercises

In [ ]:
def get_random_exercises(difficulty_level):
    body_parts = data['BodyPart'].unique()
    random_exercises = {}
    for body_part in body_parts:
        body_part_exercises = data[(data['BodyPart'] == body_part) & 
                                      (data['Level'] == difficulty_level)]['Title']
        random_exercise = body_part_exercises.sample(n=1).values[0]
        random_exercises[body_part] = random_exercise
    return random_exercises

In [ ]:
def process_input(user_input):
    if 'plan' in user_input.lower():
        level = input("Fitty: What is your fitness level (Beginner/Intermediate/Expert)? ")
        if level.lower() in ['beginner', 'intermediate', 'expert']:
            plan = get_random_exercises(level.capitalize())
            return plan
        else:
            return "Fitty: Please provide a valid fitness level."
    elif any(body_part.lower() in user_input.lower() for body_part in data['BodyPart']):
        matching_body_parts = [body_part for body_part in data['BodyPart'].unique() if body_part.lower() in user_input.lower()]
        if matching_body_parts:
            body_part = matching_body_parts[0]
            level = input(f"Fitty: What is your fitness level for {body_part}? (Beginner/Intermediate/Expert) ")
            if level.lower() in ['beginner', 'intermediate', 'expert']:
                return f"Fitty:{part_exercises(body_part,level)}"
            else:
                return "Fitty: Please provide a valid fitness level."
        else:
            return "Fitty: I'm sorry, I couldn't find any relevant body part in your query."
    elif any(exercise.lower() in user_input.lower() for exercise in data['Title']):
        matching_exercises = [exercise for exercise in data['Title'] if exercise.lower() in user_input.lower()]
        if matching_exercises:
            exercise = matching_exercises[0]
            exercise_info = data[data['Title'] == exercise].iloc[0]
            description = exercise_info['Desc']  # This line retrieves the description
            equipment = exercise_info['Equipment']
            return f"Fitty: Exercise: {exercise}\nDescription:{description}\nEquipment: {equipment}"
        else:
            return "Fitty: I'm sorry, I couldn't find any exercise matching your query."
    elif any(word in user_input.lower() for word in ['hello', 'hi', 'hey']):
        return "Fitty: Hello! I'm your personal gym assistant. How can I help you today?"
    elif 'bye' in user_input.lower():
        return "Fitty: Thank you! Goodbye."
    else:
        return f"Fitty: I'm sorry, I didn't understand that. How can I assist you?"


In [ ]:
def main():
    print(welcome_message())
    while True:
        user_input = input("User: ")
        response = process_input(user_input)
        print(response)
        if 'bye' in user_input.lower():
            break

if __name__ == "__main__":
    main()